> 킥스타터를 크롤링해 얻은 데이터를 기반으로 엑셀을 활용한 데이터 분석, 보고서 작성

# 9.1 킥스타터 API 확인하기

- 'kickstarter api'로 검색하여 'Does Kickstarter have a public API? - Stack Overflow' 문서 확인

> https://www.kickstarter.com/projects/search?item=3d+printer  
> https://www.kickstarter.com/projects/search.json?item=3d+printer

---

# 9.2 킥스타터 크롤러 만들기

- 이 API는 요청당 검색 결과를 20건만 반환한다는 점과 200쪽까지만 검색할 수 있어 총 4,000건의 데이터만 얻을 수 있다.
- 이로 인해 검색 시 기술 범주 아래의 범위로 지정
- 크롤러와 분석 코드를 분리, 크롤러는 수집 후 저장, 분석 코드는 수집된 로컬 파일로 분석

In [12]:
import urllib.request
from urllib.error import HTTPError, URLError
import json
import os
import time

os.makedirs('result', exist_ok=True)

search_term = ""
sort_key = 'newest'
category_list = [16, 331, 332, 333, 334, 335, 336, 337, 52, 362, 338, 51, 339, 340, 341, 342] # technology category
query_base = "https://www.kickstarter.com/projects/search.json?term=%s&category_id=%d&page=%d&sort=%s"

for category_id in category_list:
    for page_id in range(1, 201):
        try:
            query = query_base % (search_term, category_id, page_id, sort_key)
            print(query)
            data = urllib.request.urlopen(query).read().decode("utf-8")
            print(data)
            response_json = json.loads(data)
        except HTTPError as e:
            print("\n------------------------------------------")
            print (category_id, " : ", page_id)
            print(e)
            print('HTTPError Code : ', e.code)
            break
        except URLError as e:
            print("\n------------------------------------------")
            print (category_id, " : ", page_id)
            print("The Server Could not be found!")
            print(e)
            print('URL Error Reason : ', e.reason)
            break
            
        # 페이지당 20건의 결과가 반환되므로 1건씩 저장
        for project in response_json["projects"]:
            filepath = "result/%d.json" % project["id"]
            fp = open(filepath, "w")
            fp.write(json.dumps(project, sort_keys=True, indent=2))
            fp.close()
            
        # 접속당 1초의 대기 시간을 주어 과도한 접근을 막는다.
        time.sleep(1)

https://www.kickstarter.com/projects/search.json?term=&category_id=16&page=1&sort=newest

------------------------------------------
16  :  1
HTTP Error 403: Forbidden
HTTPError Code :  403
https://www.kickstarter.com/projects/search.json?term=&category_id=331&page=1&sort=newest

------------------------------------------
331  :  1
HTTP Error 403: Forbidden
HTTPError Code :  403
https://www.kickstarter.com/projects/search.json?term=&category_id=332&page=1&sort=newest

------------------------------------------
332  :  1
HTTP Error 403: Forbidden
HTTPError Code :  403
https://www.kickstarter.com/projects/search.json?term=&category_id=333&page=1&sort=newest

------------------------------------------
333  :  1
HTTP Error 403: Forbidden
HTTPError Code :  403
https://www.kickstarter.com/projects/search.json?term=&category_id=334&page=1&sort=newest

------------------------------------------
334  :  1
HTTP Error 403: Forbidden
HTTPError Code :  403
https://www.kickstarter.com/projects/searc

---

# 9.3 JSON 데이터를 CSV로 변환하기

- 파일 로딩 시 파이썬 표준 라이브러리의 glob 사용
- csv 형식으로의 변환에는 판다스의 pandas.io.json.json_normalize(json_data) 함수 사용

In [14]:
import glob
import json
import pandas
import pandas.io.json

project_list = []

# glob으로 result 폴더에 있는 파일을 하나씩 읽음
for filename in glob.glob("result/*.json"):
    project = json.loads(open(filename).read())
    project_list.append(project)
    
# json_normalize를 이용해 DataFrame으로 변환
df = pandas.io.json.json_normalize(project_list)

# "_at"으로 끝나는 unixtime 컬럼을 datetime으로 변환
datetime_columns = filter(lambda a: a[-3:] == "_at", df.columns)
for column in datetime_columns:
    df[column] = pandas.to_datetime(df[column], unit='s')
    
# DataFrame에서 CSV 형식의 str 형식으로 변환한다.
csv_data = df.to_csv()

# 윈도우용 엑셀에 로딩하기 위해 CP949로 변환한다.
csv_data = csv_data.encode("utf-8", "ignore")

# 결과 로딩
fp = open("kickstarter_result.csv", "wb")
fp.write(csv_data)
fp.close()

/var/folders/t4/mtkrqnk14yn1j76qhsrvnfvw0000gn/T/ipykernel_13167/3987926427.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = pandas.io.json.json_normalize(project_list)


---

# 9.4 엑셀로 데이터 살펴보기

- 가공하지 않은 데이터를 직접 보는 것은 분석 효율에 큰 영향을 준다.
- 모금액(predged)과 목표액(goal) 정보를 이용해 달성률(=pledged / goal)을, 후원자 수(backer_count)에서 1인당 후원액(=pledged / backers_count)를 계산한 컬럼을 추가
- 알 수 있는 사실들
    - 킥스타터는 종료 시점에 임박한 프로젝트를 최상위 페이지에 소개한다.
    - 종료 시점에 후원이 늘어나는 이유는 되지만 100% 근처에 특이점이 생기는 이유는 설명되지 않는다.
    - 거의 목표에 달성했을 때 프로젝트 제안자가 더 열심히 홍보한다.
    - 제안자의 지인들이 마지막 후원에 동참한다.
    - 달성이 확실한 프로젝트에만 참여하려는 사람이 후원한다.
    - 자신의 참여로 프로젝트가 달성되는 쾌감을 즐기는 사람이 있다.

---

# 9.5 피벗 테이블로 다양하게 파악하기

**엑셀 학습 필요...ㅠ.ㅠ**

---

# 9.6 목표를 달성했지만 취소된 프로젝트 확인하기

---

# 9.7 국가별로 살펴보기

---

# 9.8 보고서 작성하기

---

# 9.9 이후 작업

- 분석해볼만한 관계들
    - 프로젝트 소개글의 문장과 성공률
    - 프로젝트 소개글의 제목과 성공률
    - 팀 소개 유무와 성공률
        - 팀 소개는 프로젝트 설명에 포함되므로 자연어 처리 필요
    - 팀 구성원 수와 성공률
        - 팀 구성원 수는 팀 소개 내용에 포함되므로 자연어 처리 필요
    - 제안자의 타 프로젝트 후원 경험과 성공률
        - 킥스타터를 활용하는 사용자일수록 킥스타터의 문화를 이해하고 있어 어떤 프로젝트의 성공 가능서이 높은지 알 수 있다.
    - 프로젝트 모집 기간과 성공률
    - 프로젝트 시작일, 종료일의 요일과 성공률

---

# 9.10 정리

- 특정한 위치에 비연속적인 보상이 주어지는 유형의 문제에서는 비연속적인 보상이 생기는 구간에 착안하여 분석하는 것이 효과적이다.
- 비연속성이 존재하는 주변을 분석하면 비지니스에서 강력한 무기가 될 수 있다.